# Lecture 2.2: Simmetry aware exact diagonalization: the $J_1$-$J_2$ Heisenberg Chain

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PhilipVinc/ComputationalQuantumPhysics/blob/main/Notebooks/2-Simmetries/4-J1J2-Heisenberg-chain.ipynb)

In this exercise, we study the spin-1/2 $J_1$-$J_2$ Heisenberg chain with periodic boundary conditions:

$$
H = J_1\sum_{i=1}^N \mathbf{S}_i \cdot \mathbf{S}_{i+1} + J_2\sum_{i=1}^N \mathbf{S}_i \cdot \mathbf{S}_{i+2}
$$

where $\mathbf{S}_i = (S_i^x, S_i^y, S_i^z)$, therefore $\mathbf{S}_i \cdot \mathbf{S}_{j} = S_i^x S_j^x + S_i^y S_j^y + S_i^zS_j^z$.
We also assume periodic boundary conditions, so for simplicity we have that $\mathbf{S}_{N+1} = \mathbf{S}_1$, $\mathbf{S}_{N+2} = \mathbf{S}_2$.

This model is known under different names:
- $J_2 = 0$: Heisenberg model/chain
- $J_2/J_1 = 1/2$: This point is known as the _Majumdar-Ghosh_ point due to the particular physics it exhibits in the ground-state
- $J_2/J_1 \gtrsim 0.2411$ the ground-state is dimerized


---
## Setup

We use [QuSpin](https://quspin.github.io/QuSpin/), a Python library for exact diagonalization with built-in symmetry support.


In [ ]:
# Install QuSpin if needed (e.g., on Colab)
# !pip install numpy scipy matplotlib quspin

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.sparse.linalg import eigsh

from quspin.basis import spin_basis_1d
from quspin.operators import hamiltonian


---
# Part 1: Symmetry-Resolved Exact Diagonalization

Convince yourself that this Hamiltonian conserves the number of spin up and spin down. 
A way to do it is to rewrite it in terms of $(S^z_i,S^+_i,S^-_i)$ where $S^{\pm}_i=\frac{1}{2}\left(S^x_i\pm S^y_i\right)$ and then think a tiny bit.


## Stuff to do

- Work in the fixed $S^z_{\text{tot}} = 0$ sector using `spin_basis_1d(N, Nup=N//2)`. Compute the size of this hilbert space subsector.
- Implement translation symmetry: build momentum sectors $k = 2\pi q/N$ with `kblock=q`. Compute the size of this hilbert space subsector.
- (Optional) Add inversion/parity symmetry with `pblock=+1` or `pblock=-1`. Compute the size of this hilbert space subsector.

Show the dimension reduction
$$\text{full } 2^N \to \text{fixed } S^z \to \text{fixed } (S^z, k) \to \text{fixed parity}$$
for $N = 8, \dots, 16, \dots$ by making a plot of the Hilbert space size versus number of Spins for the different curves. Pick the best way to present this plot (linear-linear, semilogy or log-log?). Think about what this entails: did we solve the exponential growth of the hilbert space? how much did we gain?

Tip: the basis dimension is stored in `basis.Ns`.


---
# Part 2: Ground State Momentum Sectors of the Heisenberg chain

## Stuff to do

- For $J_2 = 0$, compute $E_0(k)$ for all momentum sectors for sizes $N = 8, 10, 12, 14, 16$.

    - Make a figure with many subfigures, and within each subfigure you should plot $E_0(k)$ for a given system size.
    - In each plot, identify the $k$ sub-sector where the ground-state lies. What do you see?
    - Try to identify a pattern. Where do you think is the ground state for $N=\infty$? Can you make a plot/extrapolation that proves your point?


Tip: build the Hamiltonian with
`static = [["xx", J1_list], ["yy", J1_list], ["zz", J1_list], ["xx", J2_list], ["yy", J2_list], ["zz", J2_list]]`.


In [ ]:
def build_J1J2_hamiltonian(N, J1, J2, basis):
    # Build the J1-J2 Heisenberg Hamiltonian with periodic boundary conditions
    # TODO: fill J1_list and J2_list with (coupling, i, j) entries
    # Example: J1_list = [[J1, i, (i + 1) % N] for i in range(N)]
    #          J2_list = [[J2, i, (i + 2) % N] for i in range(N)]
    static = []
    # TODO....
    # return hamiltonian...


def ground_state_energy_in_k_sectors(N, J1, J2):
    # Return arrays of momenta k and the corresponding ground-state energy E0(k)
    # TODO: loop over kblock, build basis, build H, compute lowest eigenvalue
    return None, None


In [ ]:
# TODO: compute and plot E0(k) for N = 8, 10, 12, 14, 16 at J2 = 0
# TODO: (Optional) repeat for a grid of J2/J1 values


In [ ]:
# For the multi-panel plot discussed above, you can do something like

# Plot E0(k) for each system size
fig, axes = plt.subplots(2, 3, figsize=(14, 8))
axes = axes.flatten()

for idx, N in enumerate(sizes):
    ax = axes[idx]
    k_vals =  # ... extract k value for this system size 
    E0_vals =  # ... extract E0 value for this system size 
    
    ax.plot(k_vals / np.pi, E0_vals, 'o-', markersize=8)
    
    # Highlight ground state
    gs_idx = np.nanargmin(E0_vals)
    ax.plot(k_vals[gs_idx] / np.pi, E0_vals[gs_idx], 'r*', markersize=15, label='Ground state')
    
    # labels
    ax.set_xlabel('$k/\pi$')
    ax.set_ylabel('$E_0(k)$')
    ax.set_title(f'$N = {N}$')
    ax.grid(True, alpha=0.3)
    ax.legend()


# Hide empty subplot
axes[-1].axis('off')

plt.suptitle('Ground State Energy by Momentum Sector ($J_2 = 0$)', fontsize=14)
plt.tight_layout()
plt.show()

---
# Part 3: Physical Observables

Compute one or more observables to detect the phase transition:

1. **Dimer order parameter**:
   $$D = \frac{1}{N}\sum_i (-1)^i \langle \mathbf{S}_i \cdot \mathbf{S}_{i+1}\rangle$$

2. **Spin gap (Optional)**:
   $$\Delta = E_0(S^z=1) - E_0(S^z=0)$$

3. **Static structure factor (Optional)**:
   $$S(q) = \frac{1}{N}\sum_{i,j} e^{iq(i-j)}\langle S_i^z S_j^z\rangle$$

In [ ]:
# TODO: implement one or more observables above
# Hint: you can construct correlation operators with quspin.hamiltonian and compute expectation values with psi.conj().T @ (O @ psi).


---
# Part 4 (Optional): Bipartite Entanglement Entropy

## Stuff to do

Compute the von Neumann entanglement entropy $S(\ell)$ for a bipartition of $\ell$ contiguous sites:

$$S(\ell) = -\mathrm{Tr}[\rho_A \ln \rho_A]$$

where $\rho_A$ is the reduced density matrix for subsystem $A$ (the first $\ell$ sites).

- Plot $S(\ell)$ vs $\ell$ for
  - $J_2 = 0$ (gapless)
  - $J_2/J_1 = 0.5$ (Majumdar-Ghosh point)
  - $J_2$ large (gapped)

- compare the central entropy $S(N/2)$ as a function of $N$ for two values of $J_2/J_1$.

Tip: use `basis.ent_entropy(psi, subsys_A=range(ell))`. For entanglement, use the full $S^z=0$ basis without translation symmetry.


In [ ]:
# TODO: compute the ground state in the full Sz=0 basis and evaluate entanglement entropies


---
# Optional Extensions

- Check finite-size scaling of the spin gap to estimate the critical point.
- Explore how the momentum-sector crossing sharpens with system size.
